In [1]:
import torch
import numpy as np
import pandas as pd
import seaborn as sb
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from sklearn.model_selection import train_test_split
from visdom import Visdom
%run Utility_Functions.ipynb
%run parameters.py

In [ ]:
# Data import 
x = np.load('X.npy')
y = np.load('Y.npy')

print(x.shape, y.shape, sum(y)/len(y))
print(np.min(x), np.max(x), np.mean(x))

# Visualization
viz = Visdom()
opts = dict(legend=['Training','Testing'], xlabel = 'Iteration', ylabel = 'Accuracy', title = 'EEG Classification')
# Device configuration 
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
device # verify which device is being used

In [5]:
# Prepare the data for convolution neural network 
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.30)

class custom_dataset(Dataset):
    def __init__(self, x, y):
        super(custom_dataset, self).__init__()
        self.x = torch.from_numpy(x).type(torch.float32)
        self.y = torch.from_numpy(y).type(torch.float32)
    
    def __getitem__(self, ids):
        return self.x[ids], self.y[ids] # 2 index of total force ir index
    
    def __len__(self):
        return self.x.shape[0]

# Create datasets 
train_data = custom_dataset(x_train, y_train)
test_data = custom_dataset(x_test, y_test)

def accuracy(data):
    with torch.no_grad():
        out = model(data.x.to(device))
        out = out.cpu().detach()
        accuracy = (out.argmax(dim=1)==data.y.argmax(dim=1)).float().sum()
        accuracy = accuracy/data.__len__()

        return accuracy
# Load datasets 
train_data_iterator = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

data = next(iter(train_data_iterator))

In [7]:
# Hyper-parameters 
OUTPUT = 2
LAYER_1 = 10
LAYER_2 = 10
NUM_EPOCHS = 500
BATCH_SIZE = 32
LEARNING_RATE = 0.001
# WEIGHT_DECAY = 0.0001
BIAS = True


class ConvNet(nn.Module):
    def __init__(self, LAYER_1, LAYER_2, OUTPUT):
        super(ConvNet, self).__init__()
        
        # Parameters 
        self.layer_1 = LAYER_1
        self.layer_2 = LAYER_2
        self.output = OUTPUT
        
        # Net Parameters
        self.conv_1 = self.convolution(1, 20, (5,5), BIAS)
        self.conv_2 = self.convolution(20, 1, (5,5), BIAS)
        self.conv_3 = self.convolution(1, 5, (5,5), BIAS)
        self.pool_1 = self.pooling((3,3), stride=(1,2))
        self.conv_4 = self.convolution(5, 5, (5,5), BIAS)
        self.pool_2 = self.pooling((3,3), stride=(1,2))
        self.conv_5 = self.convolution(5, 5, (5,5), BIAS)
        self.pool_3 = self.pooling((3,3), stride=(1,2))
        self.conv_6 = self.convolution(5, 5, (5,5), BIAS)
        self.pool_4 = self.pooling((3,3), stride=(2,2))
        self.conv_7 = self.convolution(5, 1, (5,5), BIAS)
        
        self.linear = nn.Sequential(nn.Linear(160, self.layer_1), nn.ReLU(), nn.Dropout(), nn.Linear(self.layer_2, self.output))
        
    def pooling(self, k_size, stride):
        pool = nn.MaxPool2d(kernel_size=k_size, stride=stride, padding=1)
        return pool
    
    def convolution(self, in_ch, out_ch, k_size, bias):
        conv = nn.Sequential(
            nn.Conv2d(in_ch, out_ch, kernel_size=k_size, stride=1, padding=2, bias=bias),
            nn.ELU()
#             nn.Dropout()
        )
        return conv        
                                   
    def forward(self, x):
        x = x.view(-1, 1, n_electrodes, epoch_length*s_freq)
        out = self.conv_1(x)
        out = self.conv_2(out)
        out = self.conv_3(out)
        out = self.pool_1(out)
        out = self.conv_4(out)
        out = self.pool_2(out)
        out = self.conv_5(out)
        out = self.pool_3(out)
        out = self.conv_6(out)
        out = self.pool_4(out) 
        out = self.conv_7(out)
        
        # Fully connected layer
        out = out.view(out.shape[0],-1)
        out = self.linear(out)
        
        return torch.sigmoid(out)
    
# An instance of model
model = ConvNet(LAYER_1, LAYER_2, OUTPUT).to(device)
viz.close(win='Accuracy')
  
# Loss and optimizer
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

i = 0

for epoch in range(NUM_EPOCHS):
    x_batch = data[0]
    y_batch = data[1]
    # Send the images and labels to gpu 
    x_batch = x_batch.to(device)
    y_batch = y_batch.to(device)
    
    # Forward pass
    out_put = model(x_batch) 
    loss = criterion(out_put, y_batch)
    
    # Backward and optimize
    optimizer.zero_grad() # For batch gradient optimisation
    loss.backward()
    optimizer.step()
        
# Calculate the accuracy
    train_accuracy = accuracy(train_data)
    test_accuracy = accuracy(test_data)

    i = i + 1 
    viz.line(X = np.column_stack([i]*2), Y = np.column_stack((train_accuracy, test_accuracy)), opts = opts, win='Accuracy', update='append')

In [ ]:
_get_n_params(model)

In [ ]:
import numpy
numpy.set_printoptions(threshold=numpy.nan)